In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os


In [2]:
train_dir = r"D:\6 SEMESTER\AIML\SBMP\Facial Detection\archive (1)\train"
test_dir  = r"D:\6 SEMESTER\AIML\SBMP\Facial Detection\archive (1)\test"


In [3]:
# Image Data Generator with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Flow from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\aditi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
model.fit(train_data, validation_data=test_data, epochs=8)

# Save the model
model.save('emotion_model.h5')
print("Model saved as emotion_model.h5 ✅")

C:\Users\aditi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 62s 134ms/step - accuracy: 0.3122 - loss: 1.7213 - val_accuracy: 0.3835 - val_loss: 1.5817
Epoch 2/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 38s 86ms/step - accuracy: 0.3822 - loss: 1.5873 - val_accuracy: 0.4373 - val_loss: 1.4472
Epoch 3/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 95ms/step - accuracy: 0.4124 - loss: 1.5211 - val_accuracy: 0.4560 - val_loss: 1.4164
Epoch 4/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.4270 - loss: 1.4852 - val_accuracy: 0.4645 - val_loss: 1.3956
Epoch 5/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.4415 - loss: 1.4523 - val_accuracy: 0.4746 - val_loss: 1.3550
Epoch 6/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 95ms/step - accuracy: 0.4524 - loss: 1.4289 - val_accuracy: 0.4953 - val_loss: 1.3236
Epoch 7/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.4595 - loss: 1.4114 - val_accuracy: 0.4886 - val_loss: 1.3172
Epoch 8/8
449/449 ━━━━━━━━━━━━━━━━━━━━ 43s 95ms/step - accuracy: 0.4705 - loss: 1.3868 - val_acc

Model saved as emotion_model.h5 ✅


In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model
model = load_model('emotion_model.h5')

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


In [ ]:
# Start webcam
cap = cv2.VideoCapture(0)

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.reshape(1, 48, 48, 1) / 255.0

        prediction = model.predict(roi)
        emotion = emotion_labels[np.argmax(prediction)]

        # Draw rectangle and label on face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    cv2.imshow('Facial Emotion Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
